In [5]:
!pip install langchain
!pip install openai
!pip install pickle
!pip install python-dotenv


[notice] A new release of pip available: 22.3 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.3 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: Could not find a version that satisfies the requirement pickle (from versions: none)
ERROR: No matching distribution found for pickle

[notice] A new release of pip available: 22.3 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.3 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [17]:
from dotenv import load_dotenv
import os

# Laden Sie die Umgebungsvariablen aus der .env-Datei
load_dotenv()
API_KEY = os.environ.get("API_KEY")

## Loaders  
Um Daten mit einem LLM zu verwenden, müssen Dokumente zunächst in eine Vectordatenbank. 
Der erste Schritt ist diese über einen Loader in memory zu laden 

In [7]:
from langchain.document_loaders import DirectoryLoader, TextLoader

loader = DirectoryLoader('./FAQ', glob="**/*.txt", loader_cls=TextLoader, show_progress=True)
docs = loader.load()

100%|██████████| 3/3 [00:00<00:00, 166.73it/s]


## Text Splitter
Texte werden nicht 1:1 in die Datenbank geladen, sondern in Stücken, sog. "Chunks". Man kann die Chunk Größe und den Overlap zwischen den Chunks definieren

In [8]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=100,
)

documents = text_splitter.split_documents(docs)
documents[0]

Document(page_content='Q: Was ist ein Huninchen?\nA: Ein Huninchen ist ein fiktives Tier, das eine Kreuzung aus einem Hund und einem Kaninchen darstellt. Es kombiniert Merkmale beider Arten.\n\nQ: Welche Farbe hat ein Huninchen?\nA: Ein Huninchen kann verschiedene Farben haben, Ã¤hnlich wie Hunde und Kaninchen. Es kann braun, weiÃŸ, schwarz, grau oder sogar eine Mischung aus diesen Farben sein.', metadata={'source': 'FAQ\\Huninchen_Basics.txt'})

## Embeddings
Texte werden nicht als Text in der Datenbank gespeichert, sondern als Vectorrepräenstation.
Embeddings sind eine Art von Wortdarstellung, die die semantische Bedeutung von Wörtern in einem Vektorraum darstellt. 

In [9]:
from langchain.embeddings import OpenAIEmbeddings
embeddings = OpenAIEmbeddings(openai_api_key=API_KEY)


## Laden der Vectoren in VectorDB (FAISS)
Wie von OpenAIEmbeddings erstellen Vectoren können nun in der Datenbank gespeichert. Die DB kann als .pkl file abgelegt werden

In [10]:
from langchain.vectorstores.faiss import FAISS
import pickle

vectorstore = FAISS.from_documents(documents, embeddings)

with open("vectorstore.pkl", "wb") as f:
    pickle.dump(vectorstore, f)

## Laden der Datenbank
Vor der Verwendung der Datenbank muss diese natürlich wieder geladen werden. 

In [11]:
with open("vectorstore.pkl", "rb") as f:
    vectorstore = pickle.load(f)

## Prompts
Bei einem LLM hat man die Möglichkeit, diesem vor einer Konversersation eine Identität zu verpassen oder zu definieren wie Frage und Antwort aussehen sollen 

In [12]:
from langchain.prompts import PromptTemplate

prompt_template = """Du bist ein Tierarzt, Usern beim Umgang mit ihrem Tier .

{context}

Question: {question}
Antwort hier:"""
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)


## Chains
Mit Chain Klassen kann man das Verhalten des LLMs leicht beeinflussen

In [13]:
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA

chain_type_kwargs = {"prompt": PROMPT}

llm = OpenAI(openai_api_key=API_KEY)
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=vectorstore.as_retriever(), chain_type_kwargs=chain_type_kwargs)

query = "Wie alt wird ein Huninhen?"
qa.run(query)


' Es kommt darauf an, welche Rasse das Huninchen ist. Einige kÃ¶nnen bis zu 10 Jahre alt werden, wÃ¤hrend andere nur 5 oder 6 Jahre leben.'

## Memory
In dem eben gezeigten Beispiel steht jede Anfrage für sich. Eine große Stärke eines LLM ist allerdings, dass diese bei einer Antwort den kompletten Chatverlauf berücksichtigen kann. Dafür muss allerdings aus den unterschiedlichen Fragen und Antworten eine Chathistorie aufgebaut werden. Mit unterschiedlichen Memory Klassen ist dies in Langchain sehr einfach.

In [14]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True, output_key='answer')

## Memory in Chains verwenden
Die Memory Klasse kann nun einfach in einer Chain verwendet werden. Erkennbar ist dies z.B. daran, dass wenn man von "es" spricht, der Bot das Huninchen in diesem Kontext versteht.

In [15]:
from langchain.chains import ConversationalRetrievalChain

qa = ConversationalRetrievalChain.from_llm(
    llm=OpenAI(model_name="text-davinci-003", temperature=0.7, openai_api_key=API_KEY),
    memory=memory,
    retriever=vectorstore.as_retriever(),
    combine_docs_chain_kwargs={'prompt': PROMPT}
)


query = "Wie alt wird ein Huninhen?"
qa({"question": query})
qa({"question": "Und wie viel frisst es?"})

{'question': 'Und wie viel frisst es?',
 'chat_history': [HumanMessage(content='Wie alt wird ein Huninhen?', additional_kwargs={}, example=False),
  AIMessage(content=' Ein Huninchen kann zwischen sechs und neun Jahren alt werden, je nachdem, wie gut es gepflegt wird.', additional_kwargs={}, example=False),
  HumanMessage(content='Und wie viel frisst es?', additional_kwargs={}, example=False),
  AIMessage(content=' Ein Huninchen sollte mindestens einmal tÃ¤glich eine kleine Menge Futter erhalten, normalerweise in Form von Hundefutter oder pflanzlichen Lebensmitteln wie Karotten und Heu. AbhÃ¤ngig von der GrÃ¶ÃŸe des Huninchens kann die tÃ¤gliche Menge Futter variieren.', additional_kwargs={}, example=False)],
 'answer': ' Ein Huninchen sollte mindestens einmal tÃ¤glich eine kleine Menge Futter erhalten, normalerweise in Form von Hundefutter oder pflanzlichen Lebensmitteln wie Karotten und Heu. AbhÃ¤ngig von der GrÃ¶ÃŸe des Huninchens kann die tÃ¤gliche Menge Futter variieren.'}

# Agents

Agenten verwenden ein LLM, um zu bestimmen, welche Aktionen in welcher Reihenfolge durchgeführt werden sollen. Eine Aktion kann entweder die Verwendung eines Werkzeugs und die Beobachtung seiner Ausgabe oder die Rückgabe an den Benutzer sein. Für die Verwendung eines Agent ist es wichtig neben dem Konzept eines LLM, ein neues Konzept und das eines "Tools" zu verstehen.

## Tools

Werkzeuge sind Funktionen, die Agenten nutzen können, um mit der Welt zu interagieren. Diese Werkzeuge können allgemeine Dienstprogramme (z. B. Suche), andere Ketten oder sogar andere Agenten sein.

In [30]:
from langchain.agents import load_tools

llm=OpenAI(model_name="text-davinci-003", temperature=0.7, openai_api_key=API_KEY)

tool_names = ["llm-math"]
tools = load_tools(tool_names, llm=llm)
tools

[Tool(name='Calculator', description='Useful for when you need to answer questions about math.', args_schema=None, return_direct=False, verbose=False, callbacks=None, callback_manager=None, func=<bound method Chain.run of LLMMathChain(memory=None, callbacks=None, callback_manager=None, verbose=False, llm_chain=LLMChain(memory=None, callbacks=None, callback_manager=None, verbose=False, prompt=PromptTemplate(input_variables=['question'], output_parser=None, partial_variables={}, template='Translate a math problem into a expression that can be executed using Python\'s numexpr library. Use the output of running this code to answer the question.\n\nQuestion: ${{Question with math problem.}}\n```text\n${{single line mathematical expression that solves the problem}}\n```\n...numexpr.evaluate(text)...\n```output\n${{Output of running the code}}\n```\nAnswer: ${{Answer}}\n\nBegin.\n\nQuestion: What is 37593 * 67?\n\n```text\n37593 * 67\n```\n...numexpr.evaluate("37593 * 67")...\n```output\n2518

In [31]:
from langchain.agents import Tool
tool_list = [
    Tool(
        name = "Mathetool",
        func=tools[0].run,
        description="Tool für LLM"
    )
]

In [32]:
from langchain.agents import initialize_agent

agent = initialize_agent(tool_list, 
                         llm, 
                         agent="zero-shot-react-description", 
                         verbose=True)
agent.run("Wie geht es dir?")



> Entering new AgentExecutor chain...
 This is not a math question.
Action: None.
Action Input: N/A
Observation: None. is not a valid tool, try another one.
Thought: I should use a tool that can answer this question.
Final Answer: I'm doing well, thank you! How about you?

> Finished chain.


"I'm doing well, thank you! How about you?"

In [36]:
agent.run("Was ist 100 geteilt durch 25?")



> Entering new AgentExecutor chain...
 Ich muss die Zahl 100 durch 25 teilen.
Action: Mathetool
Action Input: 100/25
Observation: Answer: 4.0
Thought: Ich kenne nun die Antwort.
Final Answer: 100 geteilt durch 25 ist 4.0.

> Finished chain.


'100 geteilt durch 25 ist 4.0.'

# Custom Tools

Man kann auch eigene Tools erstellen indem man eine Klasse erstellt, die von BaseTool erbt.

In [ ]:
from typing import Optional
from pydantic import BaseModel
from langchain.callbacks.manager import AsyncCallbackManagerForToolRun, CallbackManagerForToolRun

class CustomSearchTool(BaseTool):
    name = "custom_search"
    description = "useful for when you need to answer questions about current events"

    def _run(self, query: str, run_manager: Optional[CallbackManagerForToolRun] = None) -> str:
        """Use the tool."""
        return search.run(query)
    
    async def _arun(self, query: str, run_manager: Optional[AsyncCallbackManagerForToolRun] = None) -> str:
        """Use the tool asynchronously."""
        raise NotImplementedError("custom_search does not support async")